# FINAL PROJECT: SIMPLE APPROACH

You can find all the data in `../readonly/final_project_data` folder or download it from [Kaggle](https://inclass.kaggle.com/c/competitive-data-science-final-project) if you are solving locally. 

**Good luck!**

# Competition description

This challenge serves as final project for the _"How to win a data science competition"_ Coursera course.

In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - __1C Company__.

We are asking you to __predict total sales for every product and store in the next month__. By solving this competition you will be able to apply and enhance your data science skills.

### Submission File

For each id in the test set, you must predict a total number of sales. The file should contain a header and have the following format:

`ID,item_cnt_month
0,0.5
1,0.5
2,0.5
3,0.5
etc.`

### Team Limits

Three individuals.

### Submission Limits

You may submit a maximum of 5 entries per day.

### Metric

Submissions are evaluated by root mean squared error (__RMSE__). True target values are clipped into [0,20] range.

In [1]:
target_range = [0, 20]

### Leaderboard
By now lowest RMSE in PL is: `0.79215`
with `210` submissions

# Imports

In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

import os, gc, math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

from itertools import product

import data
from data import index_cols, future_index_cols
import tools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sklearn
for p in [np, pd, sklearn]:
    print (p.__name__, p.__version__)

numpy 1.13.1
pandas 0.20.3
sklearn 0.19.0


# Competition data

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

In [4]:
#DATA_FOLDER = "../input" # en kernel de kaggle
#DATA_FOLDER = "../readonly/final_project_data" # en coursera
DATA_FOLDER = "./data"
print(os.listdir(DATA_FOLDER))

['item_categories.csv', 'items.csv', 'sales_train.csv.gz', 'sample_submission.csv.gz', 'shops.csv', 'test.csv.gz', '.DS_Store', 'sample_submission.csv']


In [5]:
transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

In [6]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

# Grid preparation and initial all_data

In [7]:
all_data = data.initial_grid_and_aggregations(transactions,items,item_categories,shops)

# See
data.inspect_dataframe('ALL_DATA', all_data)

/Users/mar/miniconda3/envs/coursera-kaggle/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)



ALL_DATA (10913850, 12)
shop_id                   int32
item_id                   int32
date_block_num            int32
target                  float32
revenue                 float32
target_shop             float32
revenue_per_shop        float32
target_item             float32
revenue_per_item        float32
item_category_id          int32
target_category         float32
revenue_per_category    float32
dtype: object


,shop_id,item_id,date_block_num,target,revenue,target_shop,revenue_per_shop,target_item,revenue_per_item,item_category_id,target_category,revenue_per_category
0,0,19,0,0.0,0.0,5578.0,2966412.0,1.0,28.0,40,33489.0,8667237.0
1,0,27,0,0.0,0.0,5578.0,2966412.0,7.0,16275.0,19,8983.0,14095792.0
2,0,28,0,0.0,0.0,5578.0,2966412.0,8.0,4392.0,30,22216.0,10012800.0


,shop_id,item_id,date_block_num,target,revenue,target_shop,revenue_per_shop,target_item,revenue_per_item,item_category_id,target_category,revenue_per_category
10913847,59,22164,33,0.0,0.0,790.0,972717.0,15.0,8235.000000,37,4061.0,1.709204e+06
10913848,59,22166,33,0.0,0.0,790.0,972717.0,11.0,1650.000000,54,350.0,7.207000e+04
10913849,59,22167,33,0.0,0.0,790.0,972717.0,37.0,10874.150391,49,639.0,2.656594e+05


In [8]:
del(transactions) #no longer useful, save memory
gc.collect()

338

# MODEL

Dummy submission suggestion: using November 2014 as predicted data, e.g. date_block_num = 22 (34-12)

# SUBMISSION

## Predictions

In [11]:
# Prepare the data to predict (next month)
next_date_block_num = 34
last_block_num = 34 - 12
print(last_block_num)

22


In [15]:
data_next_month = test.copy()
#data_next_month['date_block_num'] = next_date_block_num

all_data = all_data.rename(columns={'target':'item_cnt_month'})

dummy_data = all_data[all_data['date_block_num'] == last_block_num][future_index_cols + ['item_cnt_month']]
#dummy_data.head()

data_next_month = pd.merge(data_next_month, dummy_data, on=future_index_cols, how='left').fillna(0)

data.inspect_dataframe('NEXT_MONTH', data_next_month)


NEXT_MONTH (214200, 4)
ID                  int64
shop_id             int64
item_id             int64
item_cnt_month    float32
dtype: object


,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,1.0
1,1,5,5320,0.0
2,2,5,5233,0.0


,ID,shop_id,item_id,item_cnt_month
214197,214197,45,15757,0.0
214198,214198,45,19648,0.0
214199,214199,45,969,0.0


In [16]:
    # Less combinations in test vs train
x = dummy_data[future_index_cols].copy()
y = data_next_month[future_index_cols].copy()

x = x.drop_duplicates()
y = y.drop_duplicates()

print(len(x),len(y),len(test))
print(dummy_data['item_cnt_month'].sum(), data_next_month['item_cnt_month'].sum())

316100 214200 214200
117845.0 69358.0


## Submission file

In [17]:
submission_columns = ['ID','item_cnt_month']

# Dump to file
submission_filename = './submissions/submission_dummy_lag.csv'
data_next_month[submission_columns].to_csv(submission_filename,header=True,index=False)

In [18]:
tools.head_and_tail_file(submission_filename,N=3)

ID,item_cnt_month

0,1.0

1,0.0

2,0.0

[...]

214197,0.0

214198,0.0

214199,0.0

TOTAL lines: 214201 ( 214201 )


# Future work

Because of a lack of time I have left behind some aspects that could improve the solution:

* Try an stacked model
* Use text data, encoded as n-grams or TF-IDF (more advanced, pretrained word2vec for russian language)
* Taking into account the evolution in the price of an `item_id`
* Include KNN features